In [2]:
import numpy as np
import torch
import torch.nn as nn
import collections
import time
import os
from datetime import datetime

In [4]:
# set GPU
use_cuda = torch.cuda.is_available()
print("Whether cuda can be used: {}".format(use_cuda))
device = torch.device('cuda:0' if use_cuda else 'cpu')
print(torch.cuda.get_device_name(0))

Whether cuda can be used: True
NVIDIA GeForce RTX 3080 Ti


In [4]:
# import tensorflow as tf
# print(tf.__version__)
# tf.compat.v1.disable_eager_execution()
# import tensorflow_probability as tfp

# Generating graph dataset (first test on minizipsh)

In [7]:
import networkx as nx
import json
import pickle
import collections
func_struct = collections.namedtuple('func_struct', 'func_name opt_level version compiler bb_size byte_size CFG ARCH LIB_NAME OBF bin_type file_path')
from sklearn.model_selection import train_test_split
import glob

In [23]:
def generate_dataset(input_path):
    query_dict = {}
    dedup_dict = {}
    ret_dict = {}
    num = ['0','1','2','3','4','5','6','7','8','9']
    # print(input_path) # files including minigzipsh: arch compiler oplevel
    for file in input_path:
        parse = file.split('/')[-1] # arm64-clang-O0_minigzipsh_graph.pkl
        parse = parse.split('.')[0]
        arch= parse.split('-')[0]
        compiler = parse.split('-')[1]
        last = parse.split('-')[-1]
        opt_level = last.split('_')[0]
        bin_name = last.split('_')[1]
        name_func_struct_tmp = pickle.load(open(file, 'rb'))
        for func_name in name_func_struct_tmp.keys():
            # print(func_name)
            if func_name[0] not in num:
                random_struct = {'arch': arch,
                                 'cfg': name_func_struct_tmp[func_name],
                                 'compiler': compiler,
                                 'opt': opt_level
                                }
                if func_name not in query_dict.keys():
                    query_dict[func_name] = [random_struct]
                else:
                    add_flag = True 
                    for iter in query_dict[func_name]:
                        if iter['compiler'] == random_struct['compiler'] and \
                            iter['opt'] == random_struct['opt'] and \
                            iter['cfg'].bb_size == random_struct['cfg'].bb_size and\
                            iter['arch'] == random_struct['arch'] and\
                            iter['cfg'].byte_size == random_struct['cfg'].byte_size:
                            add_flag = False
                            break 
                    if add_flag:
                        query_dict[func_name].append(random_struct)
        for key in query_dict.keys():
            dedup_dict[key] = list()
        for key in query_dict.keys():
            if key.endswith(tuple(num)):
                decom_name = key.split("_")
                decom_name = '_'.join(decom_name[:-1])
                if decom_name in query_dict.keys():
                    # sth like this vp56_rac_gets_nn_8
                    dedup_dict[decom_name] += query_dict[key]
                else:
                    dedup_dict[key] += query_dict[key]
            else:
                dedup_dict[key] += query_dict[key]
        for key in dedup_dict.keys():
            if len(dedup_dict[key])>0:
                arr = []
                fuck_tag_arr = []
                for fuck in dedup_dict[key]:
                    fuck_tag = "{}_{}_{}_{}_{}".format(fuck['arch'],
                                                    fuck['compiler'],
                                                    fuck['opt'],
                                                    str(fuck['cfg'].bb_size),
                                                    str(fuck['cfg'].byte_size))
                    if fuck_tag not in fuck_tag_arr:
                        arr.append(fuck)
                        fuck_tag_arr.append(fuck_tag)
                ret_dict[key] = arr
    
    # for func in ret_dict:
    #     print('The function {} in dataset has {}'.format(func, ret_dict[func]))
    return ret_dict

In [24]:
lib_name = ["minigzipsh"]
path = "./Dataset-graph/*" + lib_name[0] + "*.pkl"
input_path = glob.glob(path)
# print(len(input_path))
data = generate_dataset(input_path)
output_path = lib_name[0] + '_dataset.pickle'
with open(output_path, 'wb') as f_out:
    pickle.dump(data, f_out, protocol=pickle.HIGHEST_PROTOCOL)

# Test info of graph

In [20]:
input_json = r"./arm64-clang-O0_minigzipsh_graph.pkl"
def open_pkl(input_json):
    with open(input_json, 'rb') as f_in:
        info = pickle.load(f_in)
        for func_name in info.keys():
            cfg = info[func_name]
            g = cfg.CFG
            # for x in list(g.nodes.keys()):
            for x in range(1):
                # emb.append([g.node[x]['emb']])
                print(g.nodes[x]['feature'])
        
        
open_pkl(input_json)

[1, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,